In [ ]:
# (WIP) test avec bokeh

# plot wtih holoviews + datashader - bokeh with map background
!pip install holoviews
import holoviews as hv
!pip install geoviews
!pip install datashader
import geoviews as gv
import datashader as ds
from colorcet import fire, rainbow, bgy, bjy, bkr, kb, kr
from datashader.colors import colormap_select, Greys9
from holoviews.streams import RangeXY
from holoviews.operation.datashader import datashade, dynspread, rasterize
from bokeh.io import push_notebook, show, output_notebook
output_notebook()
hv.extension('bokeh')

In [ ]:
%opts Overlay [width = 800 height = 600 toolbar = 'above' xaxis = None yaxis = None]
%opts QuadMesh [tools = ['hover'] colorbar = True] (alpha = 0 hover_alpha = 0.2)

T = 0.05
PX = 1

In [ ]:
def plot_map(data, label, agg_data, agg_name, cmap):
    url = "http://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Dark_Gray_Base/MapServer/tile/{Z}/{Y}/{X}.png"
    geomap = gv.WMTS(url)
    points = hv.Points(gv.Dataset(data, kdims = ['x', 'y'], vdims = [agg_name]))
    agg = datashade(points, element_type = gv.Image, aggregator = agg_data, cmap = cmap)
    zip_codes = dynspread(agg, threshold = T, max_px = PX)
    hover = hv.util.Dynamic(rasterize(points, aggregator = agg_data, width = 50, height = 25, streams = [RangeXY]), operation = hv.QuadMesh)
    hover = hover.options(cmap = cmap)
    img = geomap * zip_codes * hover
    img = img.relabel(label)
    return img

In [ ]:
# copier df
brazil = pd.read_csv(os.path.join(path, 'data/olist_geolocation_dataset.csv'), dtype = {'geolocation_zip_code_prefix': str})

# Gets the first three and four first digits of zip codes, we will explore this further to understand how zip codes works
brazil['geolocation_zip_code_prefix_1_digits'] = brazil['geolocation_zip_code_prefix'].str[0:1]
brazil['geolocation_zip_code_prefix_2_digits'] = brazil['geolocation_zip_code_prefix'].str[0:2]
brazil['geolocation_zip_code_prefix_3_digits'] = brazil['geolocation_zip_code_prefix'].str[0:3]
brazil['geolocation_zip_code_prefix_4_digits'] = brazil['geolocation_zip_code_prefix'].str[0:4]

from datashader.utils import lnglat_to_meters as webm
x, y = webm(brazil.geolocation_lng, brazil.geolocation_lat)
brazil['x'] = pd.Series(x)
brazil['y'] = pd.Series(y)

# transforming the prefixes to int for plotting purposes
brazil['geolocation_zip_code_prefix'] = brazil['geolocation_zip_code_prefix'].astype(int)
brazil['geolocation_zip_code_prefix_1_digits'] = brazil['geolocation_zip_code_prefix_1_digits'].astype(int)
brazil['geolocation_zip_code_prefix_2_digits'] = brazil['geolocation_zip_code_prefix_2_digits'].astype(int)
brazil['geolocation_zip_code_prefix_3_digits'] = brazil['geolocation_zip_code_prefix_3_digits'].astype(int)
brazil['geolocation_zip_code_prefix_4_digits'] = brazil['geolocation_zip_code_prefix_4_digits'].astype(int)

agg_name = 'geolocation_zip_code_prefix'

#from google.colab import output
#output.enable_custom_widget_manager()

#!pip install jupyter_bokeh
plot_map(brazil, 'Zip Codes in Brazil', ds.min(agg_name), agg_name, cmap = rainbow)

# marche pas ??